In [62]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from os.path import join as pjoin
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import pearsonr, spearmanr, zscore, ttest_rel
import itertools

sys.path.append('../')
import circletrack_behavior as ctb
import circletrack_neural as ctn
import place_cells as pc
import plotting_functions as pf
import cell_overlap as co

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
## Settings
project_folder = ['MultiCon_Imaging']
experiment_folders = ['MultiCon_Imaging5', 'MultiCon_Imaging6']
dpath = f'../../{project_folder[0]}'
fig_path = f'../../../Manuscripts/MultiCon/intermediate_plots'
chance_color = 'darkgrey'
avg_color = 'midnightblue'
subject_color = '#7d7d7d'
ce_colors = ['#7A22BC', '#378616']
ce_color_dict = {'Two-context': '#378616', 'Multi-context': '#7A22BC'}
mouse_colors = ['midnightblue', 'darkred', 'darkorchid', 'darkturquoise']
male_mice = ['mc44', 'mc46', 'mc54', 'mc55']
control_mice = ['mc46', 'mc47', 'mc49', 'mc52', 'mc54', 'mc59', 'mc60']
mouse_list = ['mc44', 'mc46', 'mc49', 'mc51', 'mc52', 'mc54', 'mc55', 'mc56', 'mc58', 'mc60']
imaging5 = ['mc44', 'mc46', 'mc48', 'mc49', 'mc51', 'mc52']
group_list = ['Two-context', 'Multi-context']
session_list = [f'A{x}' for x in np.arange(1, 6)] + [f'B{x}' for x in np.arange(1, 6)] + [f'C{x}' for x in np.arange(1, 6)] + [f'D{x}' for x in np.arange(1, 6)]
control_list = [f'A{x}' for x in np.arange(1, 16)] + [f'B{x}' for x in np.arange(1, 6)]
bin_size = 0.1
velocity_thresh = 14
centroid_distance = 4
config = {'scrollZoom': True}
opacity = 0.3
data_of_interest = 'aligned_minian' ## one of behav, aligned_minian, lin_behav
np.random.seed(24601)

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

xr.set_options(keep_attrs=True)

### Interactive visualization to look at cross-registered cells between any number of sessions.

In [ ]:
## Create plot to look at cross-registered cells between any number of sessions
num_cells = 'all' ## either 'all' or some number
mouse = 'mc44'
date_list = ['2024_08_24', '2024_09_12']
day_group_str = None

if mouse in imaging5:
    crossreg_path = pjoin(dpath, f'{experiment_folders[0]}/output/cross_registration_results')
else:
    crossreg_path = pjoin(dpath, f'{experiment_folders[1]}/output/cross_registration_results')

fig = pf.custom_graph_template(x_title='Width', y_title='Height', width=600, height=600, titles=[''],
                               shared_x=True, shared_y=True)

if len(date_list) == 2:
    mappings = pd.read_pickle(pjoin(crossreg_path, f'circletrack_data/{mouse}/mappings_{centroid_distance}_{day_group_str}.pkl'))
    mappings.columns = mappings.columns.droplevel(0)
    shared_cells = mappings[date_list].dropna().reset_index(drop=True)
else:
    mappings_resolved = pd.read_pickle(pjoin(crossreg_path, f'circletrack_data/{mouse}/mappings_meta_{centroid_distance}_{day_group_str}.pkl'))
    mappings_resolved.columns = mappings_resolved.columns.droplevel(0)
    shared_cells = mappings_resolved[date_list].dropna().reset_index(drop=True)

A_shifted = xr.open_dataset(pjoin(crossreg_path, f'circletrack_data/{mouse}/A_shifted_{day_group_str}.nc'))
shiftds = xr.open_dataset(pjoin(crossreg_path, f'circletrack_data/{mouse}/shiftds_{centroid_distance}_{day_group_str}.nc'))
max_proj = shiftds['temps_shifted'].max(dim='session')

if type(num_cells) == str:
    cells_of_interest = shared_cells.copy()
else:
    neurons = np.random.choice(shared_cells[date_list[0]], num_cells)
    cells_of_interest = pd.DataFrame()
    for cell in neurons:
        ar = shared_cells[shared_cells[date_list[0]] == cell]
        cells_of_interest = pd.concat([cells_of_interest, ar], ignore_index=True)

for idx, session in enumerate(date_list):
    a = A_shifted.sel(session=session)
    cells = cells_of_interest[session]
    sub_a = a.sel(unit_id=cells.to_numpy())

    fig.add_trace(go.Heatmap(z=max_proj, colorscale='gray', showscale=False, visible=False, name=f'{session}'))
    fig.add_trace(go.Heatmap(z=sub_a['A_shifted'].mean(dim='unit_id').values, colorscale='thermal', showscale=False, 
                             opacity=opacity, showlegend=False))

fig.data[0].visible = True
steps = []
for i in np.arange(len(fig.data)):
    if i % 2 == 0:
        step = dict(
            method='update',
            args=[{'visible': [False] * len(fig.data)},
                {'title': 'Switched to: ' + date_list[int(i/2)]}],
        )
        step['args'][0]['visible'][i] = True
        step['args'][0]['visible'][i+1] = True
        steps.append(step)

sliders = [dict(
    active=0,
    steps=steps
)]

fig.update_layout(sliders=sliders)
fig['layout']['sliders'][0]['pad'] = dict(t=50)
fig.show(config=config)
# fig.write_html(pjoin(fig_path, f'slider_verification_{mouse}_all_sessions.html'))

### Plot max projection for days 1, 10, and 20.

In [ ]:
mouse = 'mc54' 
day_group_str = None
plot_data = 'max_proj'
if mouse in imaging5:
    crossreg_path = pjoin(dpath, f'{experiment_folders[0]}/output/cross_registration_results')
    session_list = ['2024_08_24', '2024_09_02', '2024_09_12']
else:
    crossreg_path = pjoin(dpath, f'{experiment_folders[1]}/output/cross_registration_results')
    session_list = ['2025_01_18', '2025_01_27', '2025_02_06']
shiftds = xr.open_dataset(pjoin(crossreg_path, f'circletrack_data/{mouse}/shiftds_{centroid_distance}_{day_group_str}.nc'))
A_shifted = xr.open_dataset(pjoin(crossreg_path, f'circletrack_data/{mouse}/A_shifted_{day_group_str}.nc'))

fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=3, shared_x=True, shared_y=True,
                               titles=['Day 1', 'Day 10', 'Day 20'], height=600, width=1300)
for idx, session in enumerate(session_list):
    if plot_data == 'spatial_footprints':
        print('Whoops')
    elif plot_data == 'max_proj':
        zdata = shiftds['temps_shifted'].sel(session=session).values / np.nanmax(shiftds['temps_shifted'].sel(session=session).values)
        zdata[np.isnan(zdata)] = 0
    fig.add_trace(go.Heatmap(x=np.arange(0, shiftds['height'].shape[0]), 
                            y=np.arange(0, shiftds['height'].shape[0]), 
                            z=zdata,
                            colorscale='gray',
                            showscale=False), row=1, col=idx+1)
    fig.update_yaxes(showgrid=False, zeroline=False, visible=False, row=1, col=idx+1)
    fig.update_xaxes(showgrid=False, zeroline=False, visible=False, row=1, col=idx+1)
fig.show(config={'scrollZoom': True})
# fig.write_image(pjoin(fig_path, f'{mouse}_example_fov_days1_10_20.png'))

### Visualize spatial footprints from any two sessions.

In [ ]:
## Create plot to look at cross-registered cells between two days
num_cells = 'all' ## either 'all' or some number
mouse = 'mc54'
date_list = ['2025_01_22']
# date_list = ['2024_08_28', '2024_08_29']
binarize_val = None
trim_val = 0.0008
if binarize_val is not None:
    colorscale = ["firebrick" if i > 0 else 'firebrick' for i in np.arange(0, 1, 0.5)]
    colorscale_two = ["midnightblue" if i > 0 else 'midnightblue' for i in np.arange(0, 1, 0.5)]
    color_list = [colorscale, colorscale_two]
else:
    if len(date_list) == 2:
        color_list = ['PuBu_r', 'Peach_r']
    else:
        color_list = ['PuBu_r']
    

if mouse in imaging5:
    crossreg_path = pjoin(dpath, f'{experiment_folders[0]}/output/cross_registration_results')
    mappings_file_str = f'mappings_{centroid_distance}.pkl'
    a_shifted_file = 'A_shifted.nc'
    shiftds_file = f'shiftds_{centroid_distance}.nc'
else:
    crossreg_path = pjoin(dpath, f'{experiment_folders[1]}/output/cross_registration_results')
    mappings_file_str = f'mappings_{centroid_distance}_None.pkl'
    a_shifted_file = 'A_shifted_None.nc'
    shiftds_file = f'shiftds_{centroid_distance}_None.nc'

fig = pf.custom_graph_template(x_title='Width', y_title='Height', width=700, height=700, titles=[''],
                               shared_x=True, shared_y=True)

A_shifted = xr.open_dataset(pjoin(crossreg_path, f'circletrack_data/{mouse}/{a_shifted_file}'))
shiftds = xr.open_dataset(pjoin(crossreg_path, f'circletrack_data/{mouse}/{shiftds_file}'))
max_proj = shiftds['temps_shifted'].max(dim='session')

for idx, session in enumerate(date_list):
    a = A_shifted.sel(session=session)
    avals = a['A_shifted'].mean(dim='unit_id').values
    if binarize_val is not None:
        avals = (avals > binarize_val).astype(float)
        zeros = np.where(avals <= binarize_val)
    else:
        zeros = np.where(avals <= trim_val)
    avals[zeros[0], zeros[1]] = np.nan

    fig.add_trace(go.Heatmap(z=avals, colorscale=color_list[idx], showscale=False, name=f'Session {idx+1}', opacity=0.6))
fig.update_layout(
    xaxis={'visible': False},
    yaxis={'visible': False}
)
fig.show()
fig.write_image(pjoin(fig_path, f'{mouse}_A5_spatialfootprints.png'), width=500, height=500)

In [ ]:
fig2 = pf.custom_graph_template(x_title='', y_title='', width=700, height=700)
fig2.add_trace(go.Heatmap(z=shiftds['temps_shifted'].sel(session=date_list[1]), colorscale='gray', showscale=False))
fig2.update_layout(xaxis={'visible': False}, yaxis={'visible': False})
fig2.show()
fig2.write_image(pjoin(fig_path, f'{mouse}_B1_maxprojection.png'))

In [ ]:
fig = pf.custom_graph_template(x_title='', y_title='')
fig.add_trace(go.Heatmap(z=avals, colorscale='gray', 
                         showscale=False, name=f'{session}', opacity=0.5))

fig.show()

In [ ]:
## Cell overlap across days, distance of x pixels between cell centers
mouse = 'mc54'

if mouse in imaging5:
    crossreg_path = pjoin(dpath, f'{experiment_folders[0]}/output/cross_registration_results')
    file_str = f'mappings_{centroid_distance}.pkl'
else:
    crossreg_path = pjoin(dpath, f'{experiment_folders[1]}/output/cross_registration_results')
    file_str = f'mappings_{centroid_distance}_None.pkl'
    
fig = pf.custom_graph_template(x_title='Day', y_title='', width=600, 
                               shared_y=True, titles=[mouse])

mappings = pd.read_pickle(pjoin(crossreg_path, f'circletrack_data/{mouse}/{file_str}'))

if mouse not in imaging5:
    mappings = mappings.drop('2025_02_07', axis=1, level=1)

overlap = co.calculate_overlap(mappings)
if mouse in imaging5:
    overlap = co.dates_to_days(overlap, '2024_08_24', days=20)
else:
    overlap = co.dates_to_days(overlap, '2025_01_18', days=20)
matrix = overlap.pivot_table(index='session_id1', columns='session_id2', values='overlap')
fig.add_trace(go.Heatmap(z=matrix.values, x=matrix.index, y=matrix.columns, coloraxis='coloraxis'))
boundaries = [5.5, 10.5, 15.5]
for boundary in boundaries:
        fig.add_vline(x=boundary, line_width=1.5, line_color='red', opacity=1)
        fig.add_hline(y=boundary, line_width=1.5, line_color='red', opacity=1) 
fig.update_yaxes(title='Day', col=1)
fig.update_layout(coloraxis_colorbar={'title': 'Overlap (%)'})
fig.show()
fig.write_image(pjoin(fig_path, f'{mouse}_overlap_heatmap.png'))

### Plot cell overlap comparing each context to every other context.

In [ ]:
# Plot of average cell overlap comparing all contexts
day_tup_list = [((0, 5), (0, 5)), ((0, 5), (6, 10)), ((0, 5), (11, 15)), ((0, 5), (16, 20)),
                ((6, 10), (0, 5)), ((6, 10), (6, 10)), ((6, 10), (11, 15)), ((6, 10), (16, 20)),
                ((11, 15), (0, 5)), ((11, 15), (6, 10)), ((11, 15), (11, 15)), ((11, 15), (16, 20)),
                ((16, 20), (0, 5)), ((16, 20), (6, 10)), ((16, 20), (11, 15)), ((16, 20), (16, 20))]
output_df = pd.DataFrame()
last_index = 0
for mouse in mouse_list:
    group = 'Control' if mouse in control_mice else 'Experimental'
    if mouse in imaging5:
        crossreg_path = pjoin(dpath, f'{experiment_folders[0]}/output/cross_registration_results')
        file_str = f'mappings_{centroid_distance}.pkl'
    else:
        crossreg_path = pjoin(dpath, f'{experiment_folders[1]}/output/cross_registration_results')
        file_str = f'mappings_{centroid_distance}_None.pkl'
        
    mappings = pd.read_pickle(pjoin(crossreg_path, f'circletrack_data/{mouse}/{file_str}'))
    if mouse not in imaging5:
        mappings = mappings.drop('2025_02_07', axis=1, level=1)
    overlap = co.calculate_overlap(mappings)
    if mouse in imaging5:
        overlap = co.dates_to_days(overlap, '2024_08_24', days=20)
    else:
        overlap = co.dates_to_days(overlap, '2025_01_18', days=20)
    matrix = overlap.pivot_table(index='session_id1', columns='session_id2', values='overlap')
    for idx, tup in enumerate(day_tup_list):
        if idx <= 3:
            context = 'A'
        elif (idx > 3) & (idx <= 7):
            context = 'B'
        elif (idx > 7) & (idx <= 11):
            context = 'C'
        elif idx > 11:
            context = 'D'
        df = co.average_overlap_across_contexts(matrix, start_tuple=tup[0], end_tuple=tup[1])
        output_df = pd.concat([output_df, df], ignore_index=True)
        output_df.loc[idx+last_index, 'base_context'] = context
        output_df.loc[idx+last_index, 'mouse'] = mouse
        output_df.loc[idx+last_index, 'group'] = group
    last_index = last_index + idx + 1
output_df['compared_to'] = ['A', 'B', 'C', 'D'] * (len(mouse_list)*4)
average_overlap = output_df.groupby(['compared_to', 'base_context', 'group'], as_index=False).agg({'avg': ['mean', 'sem']})

In [ ]:
## Plot of average cell overlap comparing all contexts for all mice
fig = pf.custom_graph_template(x_title='', y_title='Average Percent Overlap (%)', rows=1, columns=2, 
                               shared_y=True, width=800, titles=['Control', 'Experimental'])
for idx, group in enumerate(average_overlap['group'].unique()):
    gdata = average_overlap[average_overlap['group'] == group]
    for context in np.unique(gdata['compared_to']):
        plot_data = gdata[gdata['compared_to'] == context]
        if idx > 0:
            showlegend=False
        else:
            showlegend=True
        fig.add_trace(go.Scatter(x=plot_data['base_context'], y=plot_data['avg']['mean'], mode='lines+markers', name=context,
                                error_y=dict(type='data', array=plot_data['avg']['sem']), legendgroup=context, showlegend=showlegend), row=1, col=idx+1)
fig.show()
fig.write_image(pjoin(fig_path, 'overlap_between_contexts.png'))

### Plot average cell overlap comparing last days in each context.

In [ ]:
comparisons = [(5, 10), (10, 15), (15, 20)]
data = {'mouse': [], 'group': [], f'{comparisons[0][0]} to {comparisons[0][1]}': [], f'{comparisons[1][0]} to {comparisons[1][1]}': [],
        f'{comparisons[2][0]} to {comparisons[2][1]}': []}

for mouse in tqdm(mouse_list):
    group = 'Control' if mouse in control_mice else 'Experimental'
    if mouse in imaging5:
        crossreg_path = pjoin(dpath, f'{experiment_folders[0]}/output/cross_registration_results')
        file_str = f'mappings_{centroid_distance}.pkl'
    else:
        crossreg_path = pjoin(dpath, f'{experiment_folders[1]}/output/cross_registration_results')
        file_str = f'mappings_{centroid_distance}_None.pkl'
        
    mappings = pd.read_pickle(pjoin(crossreg_path, f'circletrack_data/{mouse}/{file_str}'))
    if mouse not in imaging5:
        mappings = mappings.drop('2025_02_07', axis=1, level=1)
    overlap = co.calculate_overlap(mappings)
    if mouse in imaging5:
        overlap = co.dates_to_days(overlap, '2024_08_24', days=20)
    else:
        overlap = co.dates_to_days(overlap, '2025_01_18', days=20)

    data['mouse'].append(mouse)
    data['group'].append(group)

    for comparison in comparisons:
        try:
            data[f'{comparison[0]} to {comparison[1]}'].append(overlap['overlap'][(overlap['session_id1'] == comparison[0]) & (overlap['session_id2'] == comparison[1])].values[0])
        except:
            pass
last_df = pd.DataFrame(data)
df_melt = last_df.melt(id_vars=['mouse', 'group'], var_name='comparison', value_name='overlap')

### Plot average cell overlap at every context switch. For example, compare day 4 to 5 and day 5 to 6.

In [53]:
## Bar plot of cell overlap at each context switch
last_days = [5, 10, 15]
data = {'mouse': [], 'group': [], '4 to 5': [], '5 to 6': [], '9 to 10': [], '10 to 11': [],
        '14 to 15': [], '15 to 16': []}
for mouse in tqdm(mouse_list):
    group = 'Two-context' if mouse in control_mice else 'Multi-context'
    if mouse in imaging5:
        crossreg_path = pjoin(dpath, f'{experiment_folders[0]}/output/cross_registration_results')
    else:
        crossreg_path = pjoin(dpath, f'{experiment_folders[1]}/output/cross_registration_results')
    file_str = f'mappings_{centroid_distance}_None.pkl'
        
    mappings = pd.read_pickle(pjoin(crossreg_path, f'circletrack_data/{mouse}/{file_str}'))
    if mouse not in imaging5:
        mappings = mappings.drop('2025_02_07', axis=1, level=1)
    overlap = co.calculate_overlap(mappings)
    
    if mouse in imaging5:
        overlap = co.dates_to_days(overlap, '2024_08_24', days=20)
    else:
        overlap = co.dates_to_days(overlap, '2025_01_18', days=20)

    data['mouse'].append(mouse)
    data['group'].append(group)
    for day in last_days:
        try:
            data[f'{day - 1} to {day}'].append(overlap['overlap'][(overlap['session_id1'] == day-1) & (overlap['session_id2'] == day)].values[0])
        except:
            data[f'{day - 1} to {day}'].append(np.nan)

        try:
            data[f'{day} to {day + 1}'].append(overlap['overlap'][(overlap['session_id1'] == day) & (overlap['session_id2'] == day+1)].values[0])
        except:
            data[f'{day} to {day + 1}'].append(np.nan)
            
df = pd.DataFrame(data)
df_melt = df.melt(id_vars=['mouse', 'group'], var_name='comparison', value_name='overlap')

  0%|          | 0/10 [00:00<?, ?it/s]

z:\Austin\CircleTrack\CircleTrackAnalysis\MultiCon_Imaging\..\cell_overlap.py:48: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

z:\Austin\CircleTrack\CircleTrackAnalysis\MultiCon_Imaging\..\cell_overlap.py:98: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

z:\Austin\CircleTrack\CircleTrackAnalysis\MultiCon_Imaging\..\cell_overlap.py:48: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when de

In [74]:
## A to A vs A to B
exp_df = df_melt[df_melt['group'] == 'Multi-context'].reset_index(drop=True)
cont_df = df_melt[df_melt['group'] == 'Two-context'].reset_index(drop=True)
sub_exp = exp_df[(exp_df['comparison'] == '4 to 5') | (exp_df['comparison'] == '5 to 6')]
sub_cont = cont_df[(cont_df['comparison'] == '14 to 15') | (cont_df['comparison'] == '15 to 16')]
combined = pd.concat([sub_exp, sub_cont], axis=0)
combined = combined.replace({'4 to 5': 'A to A', '5 to 6': 'A to B', '14 to 15': 'A to A', '15 to 16': 'A to B'})
avg_combined = combined.groupby(['group', 'comparison'], as_index=False).agg({'overlap': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='', y_title='', width=800, rows=1, columns=2, shared_y=True)
for idx, group in enumerate(['Two-context', 'Multi-context']):
    gdata = avg_combined[avg_combined['group'] == group]
    fig.add_trace(go.Scatter(x=gdata['comparison'], y=gdata['overlap']['mean'], mode='lines+markers', 
                             line_color=ce_color_dict[group], name=group, showlegend=True,
                             error_y=dict(type='data', array=gdata['overlap']['sem'])), row=1, col=idx+1)
for mouse in combined['mouse'].unique():
    mdata = combined[combined['mouse'] == mouse]
    mgroup = mdata['group'].unique()[0]
    col = 1 if mgroup == 'Two-context' else 2
    fig.add_trace(go.Scatter(x=mdata['comparison'], y=mdata['overlap'], mode='lines', line_color=ce_color_dict[mgroup],
                             name=mouse, line_width=1, opacity=0.6, showlegend=False), row=1, col=col)
fig.update_yaxes(title='Overlap (%)', col=1)
fig.show()
# fig.write_image(pjoin(fig_path, 'overlap_aa_ab_two_multi.png'))

In [72]:
## B to B vs B to C for any group and any comparison
comp_1_days = '14 to 15'
comp_2_days = '15 to 16' 
comp_1_context = 'C to C' 
comp_2_context = 'C to D'
group_name = 'Multi-context' 

exp_df = df_melt[df_melt['group'] == group_name].reset_index(drop=True)
sub_exp = exp_df[(exp_df['comparison'] == comp_1_days) | (exp_df['comparison'] == comp_2_days)]
sub_exp = sub_exp.replace({comp_1_days: comp_1_context, comp_2_days: comp_2_context})
avg = sub_exp.groupby(['group', 'comparison'], as_index=False).agg({'overlap': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='', y_title='Overlap (%)', width=400)
fig.add_trace(go.Scatter(x=avg['comparison'], y=avg['overlap']['mean'], mode='lines+markers', 
                            line_color=ce_color_dict[group_name], name=group_name, showlegend=False,
                            error_y=dict(type='data', array=avg['overlap']['sem'])))
for mouse in sub_exp['mouse'].unique():
    mdata = sub_exp[sub_exp['mouse'] == mouse]
    mgroup = mdata['group'].unique()[0]
    fig.add_trace(go.Scatter(x=mdata['comparison'], y=mdata['overlap'], mode='lines', line_color=ce_color_dict[mgroup],
                             name=mouse, line_width=1, opacity=0.6, showlegend=False))
fig.update_yaxes(range=[30, 65])
fig.show()
fig.write_image(pjoin(fig_path, f'{group_name}_{comp_1_context}_{comp_2_context}.png'), width=400, height=500)

In [73]:
## See if there is a significant difference in lick accuracy between certain days
groups = sub_exp['comparison'].unique()
a = sub_exp[sub_exp['comparison'] == groups[0]].reset_index(drop=True)
b = sub_exp[sub_exp['comparison'] == groups[1]].reset_index(drop=True)
ttest_rel(a['overlap'], b['overlap'], nan_policy='omit')

TtestResult(statistic=np.float64(3.1965227339128397), pvalue=np.float64(0.033010086152579235), df=np.int64(4))